<a href="https://colab.research.google.com/github/BryanCoboF/optimizacionCosecha/blob/main/optCosechaRestrSitio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuración Inicial
Llamada a librerias

In [ ]:
#Configuracion de librerias
%config IPCompleter.greedy=True
!pip install pulp
!pip install tabulate

from tabulate import tabulate
import pandas as pd
import numpy as np
import xlrd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpMinimize
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 29.1 MB/s eta 0:00:00


Carga de datos de Excel desde drive

In [ ]:

from google.colab import files
from google.colab import drive

drive.mount('/content/drive') # Permite ingresar a drive desde las carpetas de drive
df = pd.read_excel("/content/drive/MyDrive/PB/BaseOpt2024AP3.xlsx") # Carga el excel de datos de proyeccion, si se necesita cambiar el archivo se debe modificar el link



Mounted at /content/drive


Ingreso de datos deseados


1.   Volumen deseado por cada mes
3.   Densidad deseada por cada mes
2.   Edad minima y maxima deseada


In [ ]:
# Volumen deseado cada mes
vecVolDeseado = []
vecVolR11minDeseado = []
vecVolR11maxDeseado = []
vecVolmaxDeseado = []
vecDensminDeseada = []
vecDensmaxDeseada = []

#semanasMes = [5,4,4,5,4,5,4,4,5,4,4,4] #Numero de semanas por cada mes

semanasMes = [5,4,4,4,5,4,5,4,4,5,4,4] #Numero de semanas por cada mes
#Ingreso de volumen anual deseado
#volDes = float(input("Ingrese el volumen deseado 2024: "))
volDes =30000000
# Calculo del volumen deseado para cada mes en base al deseado anual y las semanas que tiene cada mes.
for i in range(12):
  vecVolDeseado.append(round((volDes/52)*semanasMes[i],0))


# --------------------------------------------------------------------------------------------------
# Solo usado para 2023 comentar para los siguientes años
#vecVolDeseado = [2700000,2700000,3375000,2700000,3375000,2700000,2700000,3375000,2700000,2700000,3375000,2700000] #Numero de semanas por cada mes
# ---------------------------------------------------------------------------------------------------

print(vecVolDeseado)
print(sum(vecVolDeseado))

#Ingreso de edad deseada
edadMinDes = float(input("Ingrese la edad minima de cosecha : "))
areaMaxCosecha = float(input("Ingrese el area máxima cosechable por sitio cada mes: "))

[2884615.0, 2307692.0, 2307692.0, 2307692.0, 2884615.0, 2307692.0, 2884615.0, 2307692.0, 2307692.0, 2884615.0, 2307692.0, 2307692.0]
29999996.0
Ingrese la edad minima de cosecha : 26
Ingrese el area máxima cosechable por sitio cada mes: 150


OBTENCION  DE DATOS DEL EXCEL
Creación de Diccionarios Y VECTORES


In [ ]:
# Obtencion de datos de sitio-año de plantacion
colSitioAnio = df['Sitio-AñoPlantación']

# Obtencion de datos area neta
colAreaNeta = df['Sum of Área Neta del Lote (Ha)']
rangosAreas = df['Sum of Área Neta del Lote (Ha)']
colAreaPlant = df['Sum of Área Total del Lote (Ha)']
colPerfilRendimiento = df['Average of Perfil Rendimiento (Sitio)']

# ------------- Obtencion de datos de edad -----------------
# Recibe data frame con los datos de cada mes del excel
# Crea un diccionario por cada mes con los datos por sitio-lote
#Enero
colEdadEne = df['Sum of Edad pond ENE']
#Febrero
colEdadFeb = df['Sum of Edad pond FEB']
#Marzo
colEdadMar = df['Sum of Edad pond MAR']
#Abril
colEdadAbr = df['Sum of Edad pond ABR']
#Mayo
colEdadMay = df['Sum of Edad pond MAY']
#Junio
colEdadJun = df['Sum of Edad pond JUN']
#Julio
colEdadJul = df['Sum of Edad pond JUL']
#Agosto
colEdadAgo = df['Sum of Edad pond AGO']
#Septiembre
colEdadSep = df['Sum of Edad pond SEP']
#Octubre
colEdadOct = df['Sum of Edad pond OCT']
#Noviembre
colEdadNov = df['Sum of Edad pond NOV']
#Diciembre
colEdadDic = df['Sum of Edad pond DIC']


#--------OBTENCION DE DATOS DE RENDIMIENTO ----------------
# Obtiene los datos de rendimiento del data frame de excel de cada mes proyectado
#Enero
colRendProyEne = df['Sum of Rend pond ENE']
#Febrero
colRendProyFeb = df['Sum of Rend pond FEB']
#Marzo
colRendProyMar = df['Sum of Rend pond MAR']
#Abril
colRendProyAbr = df['Sum of Rend pond ABR']
#Mayo
colRendProyMay = df['Sum of Rend pond MAY']
#Junio
colRendProyJun = df['Sum of Rend pond JUN']
#Julio
colRendProyJul = df['Sum of Rend pond JUL']
#Agosto
colRendProyAgo = df['Sum of Rend pond AGO']
#Septiembre
colRendProySep = df['Sum of Rend pond SEP']
#Octubre
colRendProyOct = df['Sum of Rend pond OCT']
#Noviembre
colRendProyNov = df['Sum of Rend pond NOV']
#Dicimebre
colRendProyDic = df['Sum of Rend pond DIC']


#------- OBTENCION DE DATOS DE ACCESIBILIDAD -------------------
#Enero
colAccesoEne = df['Sum of AccesibilidadENE']
#Febrero
colAccesoFeb = df['Sum of AccesibilidadFEB']
#Marzo
colAccesoMar = df['Sum of AccesibilidadMAR']
#Abril
colAccesoAbr = df['Sum of AccesibilidadABR']
#Mayo
colAccesoMay = df['Sum of AccesibilidadMAY']
#Junio
colAccesoJun = df['Sum of AccesibilidadJUN']
#Julio
colAccesoJul = df['Sum of AccesibilidadJUL']
#Agosto
colAccesoAgo = df['Sum of AccesibilidadAGO']
#Septiembre
colAccesoSep = df['Sum of AccesibilidadSEP']
#Octubre
colAccesoOct = df['Sum of AccesibilidadOCT']
#Novimebre
colAccesoNov = df['Sum of AccesibilidadNOV']
#Diciembre
colAccesoDic = df['Sum of AccesibilidadDIC']

# Obtencion de costo de activo biologico
colCostoActBio = df['Sum of CostoActBio Pond']


# -----------OBTENCION DE COSTO OPERACIONAL ------------------------
#Enero
colCostoOpeEne = df['Sum of Costo Operacional pond ENE']
#Febrero
colCostoOpeFeb = df['Sum of Costo Operacional pond FEB']
#Marzo
colCostoOpeMar = df['Sum of Costo Operacional pond MAR']
#Abril
colCostoOpeAbr = df['Sum of Costo Operacional pond ABR']
#Mayo
colCostoOpeMay = df['Sum of Costo Operacional pond MAY']
#Junio
colCostoOpeJun = df['Sum of Costo Operacional pond JUN']
#Julio
colCostoOpeJul = df['Sum of Costo Operacional pond JUL']
#Agosto
colCostoOpeAgo = df['Sum of Costo Operacional pond AGO']
#Septiembre
colCostoOpeSep = df['Sum of Costo Operacional pond SEP']
#Octubre
colCostoOpeOct = df['Sum of Costo Operacional pond OCT']
#Noviembre
colCostoOpeNov = df['Sum of Costo Operacional pond NOV']
#Diciembre
colCostoOpeDic = df['Sum of Costo Operacional pond DIC']


#----------- OBTENCION de %R8------------------------
#Enero
colPorcR8Ene = df['Sum of %R8 pond ENE']
#Febrero
colPorcR8Feb = df['Sum of %R8 pond FEB']
#Marzo
colPorcR8Mar = df['Sum of %R8 pond MAR']
#Abril
colPorcR8Abr = df['Sum of %R8 pond ABR']
#Mayo
colPorcR8May = df['Sum of %R8 pond MAY']
#Junio
colPorcR8Jun = df['Sum of %R8 pond JUN']
#Julio
colPorcR8Jul = df['Sum of %R8 pond JUL']
#Agosto
colPorcR8Ago = df['Sum of %R8 pond AGO']
#Septiembre
colPorcR8Sep = df['Sum of %R8 pond SEP']
#Octubre
colPorcR8Oct = df['Sum of %R8 pond OCT']
#Noviembre
colPorcR8Nov = df['Sum of %R8 pond NOV']
#Diciembre
colPorcR8Dic = df['Sum of %R8 pond DIC']

# ------------OBTENCION DE PROM R8 ------------------
#Enero
colPromR8Ene = df['Sum of Prom R8 pond ENE']
#Febrero
colPromR8Feb = df['Sum of Prom R8 pond FEB']
#Marzo
colPromR8Mar = df['Sum of Prom R8 pond MAR']
#Abril
colPromR8Abr = df['Sum of Prom R8 pond ABR']
#Mayo
colPromR8May = df['Sum of Prom R8 pond MAY']
#Junio
colPromR8Jun = df['Sum of Prom R8 pond JUN']
#Julio
colPromR8Jul = df['Sum of Prom R8 pond JUL']
#Agosto
colPromR8Ago = df['Sum of Prom R8 pond AGO']
#Septiembre
colPromR8Sep = df['Sum of Prom R8 pond SEP']
#Octubre
colPromR8Oct = df['Sum of Prom R8 pond OCT']
#Noviembre
colPromR8Nov = df['Sum of Prom R8 pond NOV']
#Diciembre
colPromR8Dic = df['Sum of Prom R8 pond DIC']

#----------- OBTENCION de %R11------------------------
#Enero
colPorcR11Ene = df['Sum of %R11 pond ENE']
#Febrero
colPorcR11Feb = df['Sum of %R11 pond FEB']
#Marzo
colPorcR11Mar = df['Sum of %R11 pond MAR']
#Abril
colPorcR11Abr = df['Sum of %R11 pond ABR']
#Mayo
colPorcR11May = df['Sum of %R11 pond MAY']
#Junio
colPorcR11Jun = df['Sum of %R11 pond JUN']
#Julio
colPorcR11Jul = df['Sum of %R11 pond JUL']
#Agosto
colPorcR11Ago = df['Sum of %R11 pond AGO']
#Septiembre
colPorcR11Sep = df['Sum of %R11 pond SEP']
#Octubre
colPorcR11Oct = df['Sum of %R11 pond OCT']
#Noviembre
colPorcR11Nov = df['Sum of %R11 pond NOV']
#Diciembre
colPorcR11Dic = df['Sum of %R11 pond DIC']


# ------------OBTENCION DE PROM R11 ------------------
#Enero
colPromR11Ene = df['Sum of Prom R11 pond ENE']
#Febrero
colPromR11Feb = df['Sum of Prom R11 pond FEB']
#Marzo
colPromR11Mar = df['Sum of Prom R11 pond MAR']
#Abril
colPromR11Abr = df['Sum of Prom R11 pond ABR']
#Mayo
colPromR11May = df['Sum of Prom R11 pond MAY']
#Junio
colPromR11Jun = df['Sum of Prom R11 pond JUN']
#Julio
colPromR11Jul = df['Sum of Prom R11 pond JUL']
#Agosto
colPromR11Ago = df['Sum of Prom R11 pond AGO']
#Septiembre
colPromR11Sep = df['Sum of Prom R11 pond SEP']
#Octubre
colPromR11Oct = df['Sum of Prom R11 pond OCT']
#Noviembre
colPromR11Nov = df['Sum of Prom R11 pond NOV']
#Diciembre
colPromR11Dic = df['Sum of Prom R11 pond DIC']


#Obtencion de columna de raleo
colRaleo = df['Average of Raleo']

# ---------CREACION DE MATRICES POR CADA MES DEL AÑO ------------
filas = len(colSitioAnio)  # Número de filas
columnas = 12  # Número de columnas (meses del siguiente año)


#//MATRICES//
# Inicializa las matrices n filas y 12 columnas
#Todos los valores se inicializan  con cero  /// Todas las matrices son del mismo tamaño
matrizEdad = [[0] * columnas for _ in range(filas)]
matrizRend = [[0] * columnas for _ in range(filas)]
matrizRendAux = [[0] * columnas for _ in range(filas)]
matrizAcceso = [[0] * columnas for _ in range(filas)]
matrizCostoOpe = [[0] * columnas for _ in range(filas)]
matrizCosto = [[0] * columnas for _ in range(filas)]
matrizRendPlant = [[0] * columnas for _ in range(filas)]  #Rendimiento area plantada
matrizPorcR8 = [[0] * columnas for _ in range(filas)]
matrizPromR8 = [[0] * columnas for _ in range(filas)]
matrizPorcR11 = [[0] * columnas for _ in range(filas)]
matrizPromR11 = [[0] * columnas for _ in range(filas)]
matrizAreaRestriccion = [[0] * columnas for _ in range(filas)]
matrizVolBloqueo = [[0] * columnas for _ in range(filas)]
matrizCostoBloqueo = [[0] * columnas for _ in range(filas)]


# -------------Asignacion de valores a cada mes de las matrices [n][12]---------
  # Matriz de Edades por cada mes del 2024
  # Matriz  de rendimientos
  # Matriz de Accesibilidad
  # Matriz de rendimiento de area plantada
  # Matriz de costo operacional
  # Matriz de porcentaje R8
  # Matriz de promedio R8
  # Matriz de porcentaje R11
  # Matriz de promedio R11

for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    if i == 0:
      matrizEdad[j][i] = colEdadEne[j]
      matrizRend[j][i] = colRendProyEne[j] #Rendimiento neto
      matrizAcceso[j][i] = colAccesoEne [j]
      matrizRendPlant[j][i] = colRendProyEne[j]*colAreaNeta[j]/colAreaPlant[j] #Rendimiento area plantada
      matrizCostoOpe[j][i] = colCostoOpeEne[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Ene[j]
      matrizPromR8[j][i] = colPromR8Ene[j]
      matrizPorcR11[j][i] = colPorcR11Ene[j]
      matrizPromR11[j][i] = colPromR11Ene[j]


    if i == 1:
      matrizEdad[j][i] = colEdadFeb[j]
      matrizRend[j][i] = colRendProyFeb[j]
      matrizAcceso[j][i] = colAccesoFeb [j]
      matrizRendPlant[j][i] = colRendProyFeb[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeFeb[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Feb[j]
      matrizPromR8[j][i] = colPromR8Feb[j]
      matrizPorcR11[j][i] = colPorcR11Feb[j]
      matrizPromR11[j][i] = colPromR11Feb[j]


    if i == 2:
      matrizEdad[j][i] = colEdadMar[j]
      matrizRend[j][i] = colRendProyMar[j]
      matrizAcceso[j][i] = colAccesoMar [j]
      matrizRendPlant[j][i] = colRendProyMar[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeMar[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Mar[j]
      matrizPromR8[j][i] = colPromR8Mar[j]
      matrizPorcR11[j][i] = colPorcR11Mar[j]
      matrizPromR11[j][i] = colPromR11Mar[j]


    if i == 3:
      matrizEdad[j][i] = colEdadAbr[j]
      matrizRend[j][i] = colRendProyAbr[j]
      matrizAcceso[j][i] = colAccesoAbr [j]
      matrizRendPlant[j][i] = colRendProyAbr[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeAbr[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Abr[j]
      matrizPromR8[j][i] = colPromR8Abr[j]
      matrizPorcR11[j][i] = colPorcR11Abr[j]
      matrizPromR11[j][i] = colPromR11Abr[j]


    if i == 4:
      matrizEdad[j][i] = colEdadMay[j]
      matrizRend[j][i] = colRendProyMay[j]
      matrizAcceso[j][i] = colAccesoMay [j]
      matrizRendPlant[j][i] = colRendProyMay[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeMay[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8May[j]
      matrizPromR8[j][i] = colPromR8May[j]
      matrizPorcR11[j][i] = colPorcR11May[j]
      matrizPromR11[j][i] = colPromR11May[j]


    if i == 5:
      matrizEdad[j][i] = colEdadJun[j]
      matrizRend[j][i] = colRendProyJun[j]
      matrizAcceso[j][i] = colAccesoJun [j]
      matrizRendPlant[j][i] = colRendProyJun[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeJun[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Jun[j]
      matrizPromR8[j][i] = colPromR8Jun[j]
      matrizPorcR11[j][i] = colPorcR11Jun[j]
      matrizPromR11[j][i] = colPromR11Jun[j]

    if i == 6:
      matrizEdad[j][i] = colEdadJul[j]
      matrizRend[j][i] = colRendProyJul[j]
      matrizAcceso[j][i] = colAccesoJul [j]
      matrizRendPlant[j][i] = colRendProyJul[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeJul[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Jul[j]
      matrizPromR8[j][i] = colPromR8Jul[j]
      matrizPorcR11[j][i] = colPorcR11Jul[j]
      matrizPromR11[j][i] = colPromR11Jul[j]

    if i == 7:
      matrizEdad[j][i] = colEdadAgo[j]
      matrizRend[j][i] = colRendProyAgo[j]
      matrizAcceso[j][i] = colAccesoAgo [j]
      matrizRendPlant[j][i] = colRendProyAgo[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeAgo[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Ago[j]
      matrizPromR8[j][i] = colPromR8Ago[j]
      matrizPorcR11[j][i] = colPorcR11Ago[j]
      matrizPromR11[j][i] = colPromR11Ago[j]


    if i == 8:
      matrizEdad[j][i] = colEdadSep[j]
      matrizRend[j][i] = colRendProySep[j]
      matrizAcceso[j][i] = colAccesoSep [j]
      matrizRendPlant[j][i] = colRendProySep[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeSep[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Sep[j]
      matrizPromR8[j][i] = colPromR8Sep[j]
      matrizPorcR11[j][i] = colPorcR11Sep[j]
      matrizPromR11[j][i] = colPromR11Sep[j]


    if i == 9:
      matrizEdad[j][i] = colEdadOct[j]
      matrizRend[j][i] = colRendProyOct[j]
      matrizAcceso[j][i] = colAccesoOct [j]
      matrizRendPlant[j][i] = colRendProyOct[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeOct[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Oct[j]
      matrizPromR8[j][i] = colPromR8Oct[j]
      matrizPorcR11[j][i] = colPorcR11Oct[j]
      matrizPromR11[j][i] = colPromR11Oct[j]


    if i == 10:
      matrizEdad[j][i] = colEdadNov[j]
      matrizRend[j][i] = colRendProyNov[j]
      matrizAcceso[j][i] = colAccesoNov [j]
      matrizRendPlant[j][i] = colRendProyNov[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeNov[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Nov[j]
      matrizPromR8[j][i] = colPromR8Nov[j]
      matrizPorcR11[j][i] = colPorcR11Nov[j]
      matrizPromR11[j][i] = colPromR11Nov[j]


    if i == 11:
      matrizEdad[j][i] = colEdadDic[j]
      matrizRend[j][i] = colRendProyDic[j]
      matrizAcceso[j][i] = colAccesoDic [j]
      matrizRendPlant[j][i] = colRendProyDic[j]*colAreaNeta[j]/colAreaPlant[j]
      matrizCostoOpe[j][i] = colCostoOpeDic[j]*1000 #Costo operacional por kbft
      matrizPorcR8[j][i] = colPorcR8Dic[j]
      matrizPromR8[j][i] = colPromR8Dic[j]
      matrizPorcR11[j][i] = colPorcR11Dic[j]
      matrizPromR11[j][i] = colPromR11Dic[j]

# VECTORES DE AREA NETA
vecAreaNeta = [[0] * 1 for _ in range(filas)]
vecRangosAreas = [[0] * 1 for _ in range(filas)]
vecValidaAreaRestriccion = [[0] * 1 for _ in range(filas)]
#Asignacion de valores a los vectores
for i in range(filas):
  vecAreaNeta[i] = colAreaNeta[i]
  vecRangosAreas[i] = colAreaNeta[i]
  vecValidaAreaRestriccion[i] = 0

print(vecAreaNeta)
print(vecRangosAreas)

[124.38000000000002, 8.979999999999999, 11.02, 37.92, 42.190000000000005, 35.25, 22.42, 26.509999999999998, 16.43, 23.65, 103.5, 96.51, 66.55000000000001, 64.5, 27.59, 18.26, 129.84, 77.13999999999999, 29.52, 32.44, 17.990000000000002, 27.400000000000002, 23.46, 38.75, 180.81, 0.46, 126.28, 5.98, 9.709999999999999, 40.730000000000004, 213.37, 44.56, 53.88999999999999, 78.93, 70.05, 23.49, 42.88999999999999, 123.25, 342.50000000000006, 341.25]
[124.38000000000002, 8.979999999999999, 11.02, 37.92, 42.190000000000005, 35.25, 22.42, 26.509999999999998, 16.43, 23.65, 103.5, 96.51, 66.55000000000001, 64.5, 27.59, 18.26, 129.84, 77.13999999999999, 29.52, 32.44, 17.990000000000002, 27.400000000000002, 23.46, 38.75, 180.81, 0.46, 126.28, 5.98, 9.709999999999999, 40.730000000000004, 213.37, 44.56, 53.88999999999999, 78.93, 70.05, 23.49, 42.88999999999999, 123.25, 342.50000000000006, 341.25]


Creacion del diccionario con todas las edades

RESTRICCION DE EDAD DE COSECHA Y ACCESIBILIDAD

In [ ]:
# Si el sitio cumple con los requerimientos de edad y accesibilidad en ese mes se asgina el rendimiento proyectado,
# caso contrario se asigna 0
# Pregunta primero si desea aplicar las restricciones
restEdad = 2
restAcceso = 2
restRaleo = 2

while restEdad !=0 and restEdad !=1:
  restEdad = float(input("Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : "))

while restAcceso != 0 and restAcceso !=1:
  restAcceso = float(input("Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : "))

while restRaleo != 0 and restRaleo !=1:
  restRaleo = float(input("Desea restringir Raleo? (SI: Ingrese 1,  NO: Ingrese 0) : "))


if restEdad == 1 and restAcceso == 1:
  # Restricciones completas --- Se corre solo si se desea restringir edad y acceso
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1 and matrizEdad[j][i] >= edadMinDes:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 0 and restAcceso == 1:
  # Restriccion de acceso unicamente
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizAcceso[j][i] == 1:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad== 1 and restAcceso == 0:
  # Restriccion solo de edad
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      if matrizEdad[j][i] >= edadMinDes:
        matrizRendAux[j][i] = matrizRend[j][i]
      else:
        matrizRendAux[j][i] = 0
elif restEdad == 0 and restAcceso == 0:
  # Sin restricciones
  for i in range(columnas): #recorre columnas
    for j in range(filas): # recorre filas
      matrizRendAux[j][i] = matrizRend[j][i]


#---------------Restriccion de raleo--------------------
# Si el valor de Raleo es 1 entonces se encera los rendimientos hasta julio
if restRaleo == 1:
  for j in range(columnas): #recorre columnas
      #print(j)
      for i in range(filas): # recorre filas
        if colRaleo[i] == 0:
          matrizRendAux[i][j] = matrizRendAux[i][j]
          #print("No Raleo")
        if colRaleo[i] == 1:
          if j <=5:
            #print("Raleo antes de Julio")
            matrizRendAux[i][j] = 0
          if j>5:
            #print("Raleo desde de Julio")
            matrizRendAux[i][j] = matrizRendAux[i][j]

def imprimeTablaRend():
  #Creacion de tabla de rendimientos
  numColTabla = columnas + 2
  dataRendAuxTabla = [[0] * numColTabla for _ in range(filas)] #Matriz con los datos
  titulosRendAux = ["#","Sitio-Año Plantacion", "Rend ENE", "Rend FEB", "Rend MAR", "Rend ABR", "Rend MAY", "Rend JUN", "Rend JUL", "Rend AGO", "Rend SEP", "Rend OCT", "Rend NOV", "Rend DIC"] #Titulos
  for j in range(numColTabla):
    for i in range(filas):
      if j == 0:
        dataRendAuxTabla [i][j] = i #Asignacion de numero
      if j == 1:
        dataRendAuxTabla [i][j] = colSitioAnio[i] #Asignacion de nombre de sitio
      if j>1:
        dataRendAuxTabla[i][j] = f"{int(matrizRendAux[i][j-2]):,}" #Asignacion de rendmientos
  tablaRendAux = tabulate(dataRendAuxTabla, headers=titulosRendAux, tablefmt="fancy_grid")
  print(tablaRendAux)

  #Creacion de data frame con rendimientos restringidos
  datafRendRestringido = pd.DataFrame(dataRendAuxTabla, columns=titulosRendAux)
  pathExcelRendRestr = '/content/drive/MyDrive/PB/Resultados/RendimientosRestringidos.xlsx'  # Path donde se crea el archivo resOptimizacion
  datafRendRestringido.to_excel(pathExcelRendRestr, index=False)


imprimeTablaRend()








Desea restringir edad? (SI: Ingrese 1,  NO: Ingrese 0) : 1
Desea restringir accesibilidad? (SI: Ingrese 1,  NO: Ingrese 0) : 1
Desea restringir Raleo? (SI: Ingrese 1,  NO: Ingrese 0) : 1
╒═════╤════════════════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╕
│   # │ Sitio-Año Plantacion   │ Rend ENE   │ Rend FEB   │ Rend MAR   │ Rend ABR   │ Rend MAY   │ Rend JUN   │ Rend JUL   │ Rend AGO   │ Rend SEP   │ Rend OCT   │ Rend NOV   │ Rend DIC   │
╞═════╪════════════════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│   0 │ FC-022022              │ 0          │ 0          │ 0          │ 0          │ 0          │ 0          │ 0          │ 0          │ 0          │ 0          │ 0          │ 9,523      │
├─────┼────────────────────────┼────────────┼────────────

-------------------------------------------

MODELO OPTIMIZACION


1.   Primera corrida del modelo, cosecha de haciendas grandes fijadas para cada mes




------------------------------------------------------------

In [ ]:
from pandas.core.internals.ops import operate_blockwise

#****************************** MODELO DE OPTIMIZACION *********************************************************************************

#-------------  Definicion del modelo   -----------------
#optCosecha = pulp.LpProblem('OptimizacionCosecha',pulp.LpMinimize) #Minimiza los costos
optCosecha = pulp.LpProblem('OptimizacionCosecha',pulp.LpMaximize) #Maximiza


#------------VARIABLE DE DECISION (AREA A COSECHAR)-------------------
area_a_cosechar = [[pulp.LpVariable(f'x{i}_{j}', lowBound = 0, cat='Continuous') for j in range(columnas)] for i in range(filas)]


#--------------FUNCION OBJETIVO-------------------------
#optCosecha += pulp.lpSum(area_a_cosechar[i][j] for j in range(columnas) for i in range(filas))  # area a cosechar
#optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas))  # volumen total
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j]*matrizPorcR8[i][j] for j in range(columnas) for i in range(filas))  # volumen R8
#optCosecha += pulp.lpSum(colCostoActBio[i]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas)) # costos


#--------------RESTRICCIONES----------------------------
# 1) -- Restringe las areas a cosechar de cada sitio y mes para que no sean mayores al area neta
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    optCosecha += area_a_cosechar[j][i] >= 0
    optCosecha += area_a_cosechar[j][i] <= vecAreaNeta[j]
    optCosecha += area_a_cosechar[j][i] <= areaMaxCosecha


# 2) Restringe las areas a cosechar de haciendas que estan retringidas por edad o accesibilidad (Para evitar errores en remanentes, evitar haciendas que tienen cero rendimiento y las cosecha sin dejarlas de remanente)
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    if matrizRendAux[j][i] > 0:
      optCosecha += area_a_cosechar[j][i] >= 0
    if matrizRendAux[j][i] == 0:
      optCosecha += area_a_cosechar[j][i] >=0
      optCosecha += area_a_cosechar[j][i] <= 0


# 3) -- Restringe que la suma del area a cosecha de cada sitio en todo el año sea menor o igual al area neta
for j in range(filas): # recorre filas
  optCosecha += sum(area_a_cosechar[j]) <= vecAreaNeta[j]


# 4) Restriccion de volumen anual
optCosecha += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar[i][j] for j in range(columnas) for i in range(filas)) == volDes


# 5) Restringe los volumenes de cada mes dependiendo del volumen deseado
for j in range(columnas):
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) == vecVolDeseado[j]
#---------------------------------------------------------------------------------------------------------------------------------------------------------



# 6) Restringe los costos
for j in range(columnas):
  #Restriccion de costos por kbft
  optCosecha += 1000*(sum(colCostoActBio[i]*area_a_cosechar[i][j] for i in range(filas))) >= 102*(sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)))
  optCosecha += 1000*(sum(colCostoActBio[i]*area_a_cosechar[i][j] for i in range(filas))) <= 116*(sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)))



# 7) Restriccion de promedio densidad R8 y R11
#for j in range(columnas):
  #optCosecha += sum(matrizPromR8[i][j]*area_a_cosechar[i][j] for i in range(filas)) >= 7.5 * sum(area_a_cosechar[i][j] for i in range(filas))
  #optCosecha += sum(matrizPromR8[i][j]*area_a_cosechar[i][j] for i in range(filas)) <= 8.5 * sum(area_a_cosechar[i][j] for i in range(filas))
  #optCosecha += sum(matrizPromR11[i][j]*area_a_cosechar[i][j] for i in range(filas)) >= 5.5 * sum(area_a_cosechar[i][j] for i in range(filas))
  #optCosecha += sum(matrizPromR11[i][j]*area_a_cosechar[i][j] for i in range(filas)) <= 6.5 * sum(area_a_cosechar[i][j] for i in range(filas))



# 8) Restriccion de rendimientos ponderados mensuales
rendPondMaxDes = 15900   #Rendimiento maximo deseado mensual
rendPondMinDes = 13500   #Rendimiento minimo deseado mensual
for j in range(columnas):
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) >= rendPondMinDes * sum(area_a_cosechar[i][j] for i in range(filas))
  optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) <= rendPondMaxDes * sum(area_a_cosechar[i][j] for i in range(filas))



'''
# 9) Restriccion de haciendas obligatorias
#-----------------------MUISNE-------------------------------------
optCosecha += area_a_cosechar[38][0] == 52.62   #PB-55/2020 Enero
optCosecha += area_a_cosechar[38][1] == 52.64   #PB-55/2020 Febrero
optCosecha += area_a_cosechar[39][0] == 11   #PB-56/2020 Enero
optCosecha += area_a_cosechar[39][1] == 7.65   #PB-56/2020 Febrero
optCosecha += area_a_cosechar[40][0] == 11   #PB-56/2021 Enero
optCosecha += area_a_cosechar[40][1] == 11.68   #PB-56/2021 Febrero
#------------------------------------------------------------------
'''



for i in range(6):
  optCosecha += area_a_cosechar[10][i+6] == vecAreaNeta[10]/6 # MS09 en 2do semestre



#Restriccion muisne todo el año
for i in range(columnas):
  optCosecha += area_a_cosechar[30][i] == vecAreaNeta[30]/12
  optCosecha += area_a_cosechar[31][i] == vecAreaNeta[31]/12
  optCosecha += area_a_cosechar[32][i] == vecAreaNeta[32]/12
  optCosecha += area_a_cosechar[33][i] == vecAreaNeta[33]/12
  optCosecha += area_a_cosechar[34][i] == vecAreaNeta[34]/12




# 10) Restringe el perfil de rendimiento de las haciendas
#for j in range(columnas):
  #optCosecha += sum(area_a_cosechar[i][j]*colPerfilRendimiento[i] for i in range(filas)) >= 1.9 * sum(area_a_cosechar[i][j] for i in range(filas))



#Resolucion
optCosecha.solve()

#****************************** FIN MODELO*********************************************************************************

#--------------Impresion de resultados preliminares--------------------------------------------------------------------
a=1
print("Areas a cosechar cada mes por sitio")

#volumenBloqueoStr = f"{int(volumenBloqueo):,}"

def imprimeAreasResOpt1():
  numColAreaResOpt1 = columnas + 2
  dataAreaResOpt1Tabla = [[0] * numColAreaResOpt1 for _ in range(filas)]
  titulosAreaResOpt1 = [" # ","Sitio-Año Plantacion", "Area ENE", "Area FEB", "Area MAR", "Area ABR", "Area MAY", "Area JUN", "Area JUL", "Area AGO", "Area SEP", "Area OCT", "Area NOV", "Area DIC"]

  for j in range(numColAreaResOpt1):
    for i in range(filas):
      if j == 0:
        dataAreaResOpt1Tabla [i][j] = i+1
      if j == 1:
        dataAreaResOpt1Tabla [i][j] = colSitioAnio[i]
      if j > 1:
        dataAreaResOpt1Tabla [i][j] = round(area_a_cosechar[i][j-2].value(),1)

  tablaAreaResOpt1 = tabulate(dataAreaResOpt1Tabla, headers=titulosAreaResOpt1, tablefmt="fancy_grid")
  print(tablaAreaResOpt1)

imprimeAreasResOpt1()


 #f"{int(matrizRendAux[i][j-1]):,}"


print("Volumen Resultante por mes: ")
for b in range(columnas):
  #Imprime el volumen a entregar cada mes

  print("Vol> ", b+1, "->", f"{int(sum(matrizRendAux[i][b]*area_a_cosechar[i][b].value() for i in range(filas) )):,}")

#Imprime el volumen total anual
print("Volumen Anual: ",f"{int(sum(matrizRendAux[i][j]*area_a_cosechar[i][j].value() for j in range(columnas) for i in range(filas))):,}")
#-----------------------------------------------------------------------------------------------------------------------



Areas a cosechar cada mes por sitio
╒═══════╤════════════════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╕
│    #  │ Sitio-Año Plantacion   │   Area ENE │   Area FEB │   Area MAR │   Area ABR │   Area MAY │   Area JUN │   Area JUL │   Area AGO │   Area SEP │   Area OCT │   Area NOV │   Area DIC │
╞═══════╪════════════════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│     1 │ FC-022022              │        0   │        0   │        0   │        0   │        0   │        0   │        0   │        0   │        0   │        0   │        0   │        0   │
├───────┼────────────────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│     2 │

Correccion de errores

In [ ]:

vecMeses = ["ENE: ", "FEB: ", "MAR: ", "ABR: ", "MAY: ", "JUN: ", "JUL: ", "AGO: ", "SEP: ", "OCT: ", "NOV: ", "DIC: "]
# Matriz que obtiene los resultados de las areas optimizadas de cosecha
colRes = columnas + 1
matrizAreaRes = [[0] * columnas for _ in range(filas)]
matrizResultados = [[0] * colRes for _ in range(filas)] #Matriz para el df
vecPosErroresArea = []
vecPosErroresNeg = []
#Asignacion de resultados de area e impresion de errores
#Impresion de errores
countErroresNeg = 0
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    matrizAreaRes [j][i] = area_a_cosechar[j][i].value()
    if area_a_cosechar[j][i].value() <-0.001:
      countErroresNeg = countErroresNeg + 1
      vecPosErroresNeg.append([j+1,i])
print("Errores negativos-> ", countErroresNeg)
print("Posiciones de errores", vecPosErroresNeg)
countErroresArea = 0
for i in range(filas):
  areaCos = sum(matrizAreaRes [i][j] for j in range(columnas))
  if areaCos > vecAreaNeta[i]+1:
    countErroresArea = countErroresArea + 1
    vecPosErroresArea.append(i)

print("Errores de area ->", countErroresArea)
print("Posiciones de errores", vecPosErroresArea)


#Titulos
titulosAreasRestringidas1 = ['Sitio-Año de plantación', 'Área ENE 2024', 'Área FEB 2024','Área MAR 2024', 'Área ABR 2024','Área MAY 2024','Área JUN 2024','Área JUL 2024','Área AGO 2024', 'Área SEP 2024','Área OCT 2024','Área NOV 2024','Área DIC 2024']


for j in range(columnas+1):
  for i in range(filas):
    if j==0:
      #Asigna los nombres de cada sitio-año de plantacion
      matrizResultados[i][j] = colSitioAnio[i]
    if j>=1:
      #Asigna las areas restringidas
      matrizResultados[i][j] = matrizAreaRes [i][j-1]

dataf = pd.DataFrame(matrizResultados, columns=titulosAreasRestringidas1)
# Path donde se crea el archivo resOptimizacion
pathExcel1 = '/content/drive/MyDrive/PB/Resultados/resOptimizacion.xlsx'
#Guarda el data frame en el excel
dataf.to_excel(pathExcel1, index=False)



#Vectores de resultados por mes
volTotalRes = []
areaTotalRes = []
rendTotalRes = []
costoTotalRes = []
vecCostoTotalResOpe = []
volTotalR8Res = []
volTotalR11Res = []
promR8Res = []
promR11Res = []

#------------CALCULO DE RESULTADOS POR MES -----------------------
#calculo de costos resultantes por mes
for j in range(columnas):
  costoTotalRes1 = 1000*(sum(colCostoActBio[i]*matrizAreaRes[i][j] for i in range(filas))/sum(matrizRendAux[i][j]* matrizAreaRes[i][j] for i in range(filas)))
  costoTotalRes.append(round(costoTotalRes1,1))
  #costoTotalResOpe = sum(((colCostoActBio[i]*matrizAreaRes[i][j])+matrizCostoOpe[i][j])/(matrizRendAux[i][j]*matrizAreaRes[i][j]/1000) for i in range(filas) if matrizRendAux[i][j]>0)
  #vecCostoTotalResOpe.append(round(costoTotalResOpe,1))

#Calculo de volumenes (total, R8, y R11) y area resultantes por mes
for j in range(columnas):
  # Volumen total por mes
  volTotal = sum(matrizRendAux[i][j]*matrizAreaRes[i][j] for i in range(filas))
  volTotalRes.append(round(volTotal,0))
  #Area Total por mes
  areaTotal = sum(matrizAreaRes[i][j] for i in range(filas))
  areaTotalRes.append(round(areaTotal,1))
  #Volumen R8 total por mes
  volTotalR8 = sum(matrizRendAux[i][j]*matrizAreaRes[i][j]*matrizPorcR8[i][j] for i in range(filas))
  volTotalR8Res.append(round(volTotalR8,0))
  #Volumen R11 total por mes
  volTotalR11 = sum(matrizRendAux[i][j]*matrizAreaRes[i][j]*matrizPorcR11[i][j] for i in range(filas))
  volTotalR11Res.append(round(volTotalR11,0))
  # Promedio R8 resultante por mes
  promR8 = sum(matrizAreaRes[i][j]*matrizPromR8[i][j] for i in range(filas))
  promR8Res.append(round(promR8/areaTotal,1))
  # Promedio R11 resultante por mes
  promR11 = sum(matrizAreaRes[i][j]*matrizPromR11[i][j] for i in range(filas))
  promR11Res.append(round(promR11/areaTotal,1))


#Rendimiento resultante por mes
for k in range(len(volTotalRes)):
  if volTotalRes[k]>0:
    rendTotalRes.append(round(volTotalRes[k]/areaTotalRes[k],1))
  else:
    rendTotalRes.append(0)

#costoBloqueadoStr = f"{int(costoBloqueo):,}"

#Presentacion de resultados
print("----------------------------------")
print('Volumen Anual Deseado')
volAnualDesStr = f"{int(sum(vecVolDeseado)):,}"   # Volumen anual Deseado Formato de str con separacion de comas
print(volAnualDesStr)

print("Volumen anual Resultante")
volTotalResStr = f"{int(sum(volTotalRes)):,}"   # Volumen anual Resultante Formato de str con separacion de comas
print(volTotalResStr)

print('Volumen mensual deseado')


volMensualDesStr = []
for i in range(columnas):
  volMensualDesStr.append(f"{int(vecVolDeseado[i]):,}")
  volMensualDesStr[i] =  vecMeses[i] + volMensualDesStr[i]

print(volMensualDesStr)

print("Volumen Resultante por mes")
print(volTotalRes)

print("Volumen R8 por mes")
print(volTotalR8Res)

print("Promedio R8 por mes")
print(promR8Res)

print("Volumen R11 por mes")
print(volTotalR11Res)

print("Promedio R11 por mes")
print(promR11Res)

print("Area a cosechar por mes")
print(areaTotalRes)

print("Costo kbft por mes")
print(costoTotalRes)
#print("Costo kbft por mes INCLUYE OPERACIONAL")
#print(costoTotalResOpe)

print("Rendimiento por mes")
print(rendTotalRes)
print("----------------------------------")


# Remanentes
for j in range(filas): # recorre filas
  if round(sum(matrizAreaRes[j]),2)<= round(colAreaNeta[j],2)-1:
    print(j, " - ",colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2), "-- Remanente 2025 -- ",round(colAreaNeta[j],2)-round(sum(matrizAreaRes[j]),2))
  else:
    print(j, " - ",colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2))



Errores negativos->  0
Posiciones de errores []
Errores de area -> 0
Posiciones de errores []
----------------------------------
Volumen Anual Deseado
29,999,996
Volumen anual Resultante
30,000,000
Volumen mensual deseado
['ENE: 2,884,615', 'FEB: 2,307,692', 'MAR: 2,307,692', 'ABR: 2,307,692', 'MAY: 2,884,615', 'JUN: 2,307,692', 'JUL: 2,884,615', 'AGO: 2,307,692', 'SEP: 2,307,692', 'OCT: 2,884,615', 'NOV: 2,307,692', 'DIC: 2,307,692']
Volumen Resultante por mes
[2884615.0, 2307692.0, 2307692.0, 2307696.0, 2884615.0, 2307692.0, 2884615.0, 2307692.0, 2307692.0, 2884615.0, 2307692.0, 2307692.0]
Volumen R8 por mes
[1145117.0, 689824.0, 762192.0, 905241.0, 1172786.0, 925401.0, 1345268.0, 1073707.0, 1070467.0, 1341261.0, 1068096.0, 1114275.0]
Promedio R8 por mes
[8.3, 8.0, 8.1, 8.3, 8.3, 8.4, 8.8, 8.8, 8.8, 8.8, 8.9, 9.1]
Volumen R11 por mes
[1739287.0, 1617752.0, 1545370.0, 1402297.0, 1711624.0, 1382110.0, 1538752.0, 1233371.0, 1236693.0, 1542695.0, 1239054.0, 1192587.0]
Promedio R11 por me

Inicio 2da Ejecución del modelo
- Preparacion de matriz de restricciones de area segun las haciendas optimizadas en la primera ejecucion.

- En base a la solución optimizada inicial crea una matriz de areas resultantes, en las cuales se fijará las haciendas grandes desde el primer mes que entregó la cosecha optimizada inicial.

In [ ]:
#Obtencion de categorias de tamaño de las haciendas por su area
for i in range(filas):
  if vecAreaNeta[i] <= 25:     #Muy Pequeña
    vecRangosAreas[i] = "Muy pequeña"
  elif vecAreaNeta[i] <= 50:   #Pequeña
    vecRangosAreas[i] = "Pequeña"
  elif vecAreaNeta[i] <= 100:  #Mediana
    vecRangosAreas[i] = "Mediana"
  elif vecAreaNeta[i] <= 150:  #Media+
    vecRangosAreas[i] = "Media +"
  elif vecAreaNeta[i] <= 200:  #Grande
    vecRangosAreas[i] = "Grande"
  elif vecAreaNeta[i] <= 300:  #Muy Grande
    vecRangosAreas[i] = "Muy Grande"
  elif vecAreaNeta[i] > 300:   #Gigante
    vecRangosAreas[i] = "Gigante"


#--------------Impresion de resultados preliminares--------------------------------------------------------------------
a=1
for j in range(filas): # recorre filas
  # Imprime cada sitio con el area asignada en el mes que se va a cosechar
  print(a, ") Areas ENE ", colSitioAnio[j], "-> " ,round(area_a_cosechar[j][0].value(),1), ", " ,round(area_a_cosechar[j][1].value(),1), ", " ,round(area_a_cosechar[j][2].value(),1),", " ,round(area_a_cosechar[j][3].value(),1),", " ,round(area_a_cosechar[j][4].value(),1),", " ,round(area_a_cosechar[j][5].value(),1),", " ,round(area_a_cosechar[j][6].value(),1),", " ,round(area_a_cosechar[j][7].value(),1), ", " ,round(area_a_cosechar[j][8].value(),1),", " ,round(area_a_cosechar[j][9].value(),1),", " ,round(area_a_cosechar[j][10].value(),1),", " ,round(area_a_cosechar[j][11].value(),1))
  a = a+1


for b in range(columnas):
  #Imprime el volumen a entregar cada mes
  print("Vol> ", b+1, "->", round(sum(matrizRendAux[i][b]*area_a_cosechar[i][b].value() for i in range(filas) ),0))


#Imprime el volumen total anual
print(sum(matrizRendAux[i][j]*area_a_cosechar[i][j].value() for j in range(columnas) for i in range(filas) ))
#-----------------------------------------------------------------------------------------------------------------------



#Impresion de Remanentes
for j in range(filas): # recorre filas
  if round(sum(matrizAreaRes[j]),2)<= round(colAreaNeta[j],2)-1:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2), "-- Remanente 2025 -- ",round(colAreaNeta[j],2)-round(sum(matrizAreaRes[j]),2))
  else:
    print(colSitioAnio[j], "-- ", round(sum(matrizAreaRes[j]),2),"--", round(colAreaNeta[j],2))

print("--------------------------------------------------------------------------------------------------------------")


# El vector permite que solo se bloquee una vez la hacienda que esta en el rango deseado para el bloqueo
for i in range(filas):
  vecValidaAreaRestriccion[i] = 0


#Encera la matriz de restricciones
for j in range(columnas):
  for i in range(filas):
    matrizAreaRestriccion[i][j] = 0


#--------------------Identificacion de primer mes de cosecha optimizado---------------------------------------------
matrizPrimerMesCosechado = [[0] * columnas for _ in range(filas)] #Identifica el primer mes en que se cosecha cada hacienda
# 0 si no encontro el primer mes cosechado o si no se cosecho la hacienda / 1 si ya encontro el primer mes cosechado
vecValidaMesCosechado = []
vecDeseaRestringir = []
vecMesesRestriccion = []
# Encera los valores del vector de validacion de primer mes cosechado
for i in range(filas):
  vecValidaMesCosechado.append(0)
  vecDeseaRestringir.append(2)
  vecMesesRestriccion.append(-1)


#Inicializa en 0 la matriz de primer mes cosechado
for j in range(columnas):
  for i in range(filas):
    matrizPrimerMesCosechado[i][j] = 0

#Identificacion de primer mes cosechado por sitio
for j in range(columnas):
  for i in range(filas):
    if  matrizAreaRes[i][j]>=0.2 and matrizPrimerMesCosechado[i][j] == 0 and vecValidaMesCosechado [i] == 0:
      matrizPrimerMesCosechado[i][j] = 1
      vecValidaMesCosechado [i] = 1

#print("Primer mes cosechado por hacienda")
#for i in range(filas):
  #print(colSitioAnio[i],matrizPrimerMesCosechado[i],vecValidaMesCosechado[i])


# Presentacion de resultados de bloqueo por mes cada que se bloquea una hacienda
volBloqueado = [0]*columnas
costoBloqueado = [0]*columnas

def valoresBloqueo():
  # Calcula los resultados de bloqueo despues de que se bloquea cada hacienda
  for n in range(columnas):
    # Calculo de volumen bloqueado por mes
    volumenBloqueo = sum(matrizAreaRestriccion[m][n] * matrizRendAux[m][n] for m in range(filas))
    volumenBloqueoStr = f"{int(volumenBloqueo):,}"
    # Concatena el mes a los volumenes resultantes
    if n == 0:
      volumenBloqueoStr = "ENE: " + volumenBloqueoStr
    if n == 1:
      volumenBloqueoStr = "FEB: " + volumenBloqueoStr
    if n == 2:
      volumenBloqueoStr = "MAR: " + volumenBloqueoStr
    if n == 3:
      volumenBloqueoStr = "ABR: " + volumenBloqueoStr
    if n == 4:
      volumenBloqueoStr = "MAY: " + volumenBloqueoStr
    if n == 5:
      volumenBloqueoStr = "JUN: " + volumenBloqueoStr
    if n == 6:
      volumenBloqueoStr = "JUL: " + volumenBloqueoStr
    if n == 7:
      volumenBloqueoStr = "AGO: " + volumenBloqueoStr
    if n == 8:
      volumenBloqueoStr = "SEP: " + volumenBloqueoStr
    if n == 9:
      volumenBloqueoStr = "OCT: " + volumenBloqueoStr
    if n == 10:
      volumenBloqueoStr = "NOV: " + volumenBloqueoStr
    if n == 11:
      volumenBloqueoStr = "DIC: " + volumenBloqueoStr
    volBloqueado[n] = volumenBloqueoStr

  for n in range(columnas):
    # Calculo de costos bloqueados por mes
    costoBloqueo = 0
    costoAreaBloqueo = 0
    rendAreaBloqueo = 0
    for m in range(filas):
      #Calculos por mes
      costoAreaBloqueo = costoAreaBloqueo + (colCostoActBio[n] * matrizAreaRestriccion[m][n]) #Costo en $
      rendAreaBloqueo = rendAreaBloqueo + (matrizAreaRestriccion[m][n] * matrizRendAux [m][n])  # Volumen en bft
    if rendAreaBloqueo > 0:
      costoBloqueo = 1000*(costoAreaBloqueo / rendAreaBloqueo) # Costo por bft ( $ / bft )
      costoBloqueadoStr = f"{int(costoBloqueo):,}"
      # Concatena el mes a los volumenes resultantes
      if n == 0:
        costoBloqueadoStr = "ENE: $" + costoBloqueadoStr
      if n == 1:
        costoBloqueadoStr = "FEB: $" + costoBloqueadoStr
      if n == 2:
        costoBloqueadoStr = "MAR: $" + costoBloqueadoStr
      if n == 3:
        costoBloqueadoStr = "ABR: $" + costoBloqueadoStr
      if n == 4:
        costoBloqueadoStr = "MAY: $" + costoBloqueadoStr
      if n == 5:
        costoBloqueadoStr = "JUN: $" + costoBloqueadoStr
      if n == 6:
        costoBloqueadoStr = "JUL: $" + costoBloqueadoStr
      if n == 7:
        costoBloqueadoStr = "AGO: $" + costoBloqueadoStr
      if n == 8:
        costoBloqueadoStr = "SEP: $" + costoBloqueadoStr
      if n == 9:
        costoBloqueadoStr = "OCT: $" + costoBloqueadoStr
      if n == 10:
        costoBloqueadoStr = "NOV: $" + costoBloqueadoStr
      if n == 11:
        costoBloqueadoStr = "DIC: $" + costoBloqueadoStr
      costoBloqueado[n] = costoBloqueadoStr
    else:
      if n == 0:
        costoBloqueadoStr = "ENE: $0"
      if n == 1:
        costoBloqueadoStr = "FEB: $0"
      if n == 2:
        costoBloqueadoStr = "MAR: $0"
      if n == 3:
        costoBloqueadoStr = "ABR: $0"
      if n == 4:
        costoBloqueadoStr = "MAY: $0"
      if n == 5:
        costoBloqueadoStr = "JUN: $0"
      if n == 6:
        costoBloqueadoStr = "JUL: $0"
      if n == 7:
        costoBloqueadoStr = "AGO: $0"
      if n == 8:
        costoBloqueadoStr = "SEP: $0"
      if n == 9:
        costoBloqueadoStr = "OCT: $0"
      if n == 10:
        costoBloqueadoStr = "NOV: $0"
      if n == 11:
        costoBloqueadoStr = "DIC: $0"
      costoBloqueado[n] = costoBloqueadoStr

  print("***********************************************")
  print("Volumen por mes bloqueado")
  print(volBloqueado)
  print("Costo por mes bloqueado")
  print(costoBloqueado)
  print("***********************************************")



#------------------------Inicio de definicion de restricciones de areas por sitio--------------------------------
for i in range(filas):
  # Recorre cada Sitio-AnioPlantacion
  if vecValidaMesCosechado [i] == 0:
    #Sitios que no cosecho el modelo
    print(colSitioAnio[i], " - No Cosechado en 2024")


  if vecValidaMesCosechado [i] == 1:
    #Sitios que cosecho el modelo en Enero
    # ******Bloqueos para Enero*********************************************************************
    if matrizPrimerMesCosechado [i][0] == 1:
      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][0])
      print("Desea restringir ", colSitioAnio[i], " desde Enero ")
      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Enero")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Enero 1 mes
              matrizAreaRestriccion [i][0] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1


            if vecMesesRestriccion [i] == 2: # Enero 2 meses
              matrizAreaRestriccion [i][0] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Enero 3 meses
              matrizAreaRestriccion [i][0] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: # Enero 4 meses
              matrizAreaRestriccion [i][0] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Enero 5 meses
              matrizAreaRestriccion [i][0] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Enero 6 meses
              matrizAreaRestriccion [i][0] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")
    #**********************************************************************************************


    # ******Bloqueos para Febrero***************
    if matrizPrimerMesCosechado [i][1] == 1:
      print(colSitioAnio [i]," - Cosechado en Febrero ")
      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][1])
      print("Desea restringir ", colSitioAnio[i], " desde Febrero ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Febrero")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Febrero 1 mes
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Febrero 2 meses
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Febrero 3 meses
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: # febrero 4 meses
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Febrero 5 meses
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Febrero 6 meses
              matrizAreaRestriccion [i][1] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")

    #**********************************************************************************************


    # ******Bloqueos para Marzo***************
    if matrizPrimerMesCosechado [i][2] == 1:
      print(colSitioAnio [i]," - Cosechado en Marzo ")
      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][2])
      print("Desea restringir ", colSitioAnio[i], " desde Marzo ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Marzo")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Marzo 1 mes
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Marzo 2 meses
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Marzo 3 meses
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Marzo 4 meses
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Marzo 5 meses
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Marzo 6 meses
              matrizAreaRestriccion [i][2] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")


    #**********************************************************************************************

    # ******Bloqueos para Abril***************
    if matrizPrimerMesCosechado [i][3] == 1:
      print(colSitioAnio [i]," - Cosechado en Abril ")
      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][3])
      print("Desea restringir ", colSitioAnio[i], " desde Abril ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Abril")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Abril 1 mes
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Abril 2 meses
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Abril 3 meses
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Abril 4 meses
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Abril 5 meses
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Abril 6 meses
              matrizAreaRestriccion [i][3] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")

    #**********************************************************************************************


    # ******Bloqueos para Mayo***************
    if matrizPrimerMesCosechado [i][4] == 1:
      print(colSitioAnio [i]," - Cosechado en Mayo ")
      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][4])
      print("Desea restringir ", colSitioAnio[i], " desde Mayo ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Mayo")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Mayo 1 mes
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Mayo 2 meses
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Mayo 3 meses
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Mayo 4 meses
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Mayo 5 meses
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Mayo 6 meses
              matrizAreaRestriccion [i][4] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")

    #**********************************************************************************************


    # ******Bloqueos para Junio***************
    if matrizPrimerMesCosechado [i][5] == 1:
      print(colSitioAnio [i]," - Cosechado en Junio ")
      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][5])
      print("Desea restringir ", colSitioAnio[i], " desde Junio ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Junio")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Junio 1 mes
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Junio 2 meses
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Junio 3 meses
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Junio 4 meses
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Junio 5 meses
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Junio 6 meses
              matrizAreaRestriccion [i][5] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")


    #**********************************************************************************************


    # ******Bloqueos para Julio***************
    if matrizPrimerMesCosechado [i][6] == 1:
      print(colSitioAnio [i]," - Cosechado en Julio ")
      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][6])
      print("Desea restringir ", colSitioAnio[i], " desde Julio ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Julio")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Julio 1 mes
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Julio 2 meses
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Julio 3 meses
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Julio 4 meses
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Julio 5 meses
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Julio 6 meses
              matrizAreaRestriccion [i][6] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")


    #**********************************************************************************************


    # ******Bloqueos para Agosto***************
    if matrizPrimerMesCosechado [i][7] == 1:
      print(colSitioAnio [i]," - Cosechado en Agosto ")

      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][7])
      print("Desea restringir ", colSitioAnio[i], " desde Agosto ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Agosto")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Agosto 1 mes
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Agostoo 2 meses
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Agosto 3 meses
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Agosto 4 meses
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Agosto 5 meses
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Agosto 6 meses
              matrizAreaRestriccion [i][7] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i])
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")

    #**********************************************************************************************


    # ******Bloqueos para Septiembre***************
    if matrizPrimerMesCosechado [i][8] == 1:
      print(colSitioAnio [i]," - Cosechado en Septiembre ")

      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][4])
      print("Desea restringir ", colSitioAnio[i], " desde Septiembre ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Septiembre")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Septiembre 1 mes
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Septiembre 2 meses
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Septiembre 3 meses
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Septiembre 4 meses
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Septiembre 5 meses
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i])
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Septiembre 6 meses
              matrizAreaRestriccion [i][8] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*2)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")

    #**********************************************************************************************



    # ******Bloqueos para Octubre***************
    if matrizPrimerMesCosechado [i][9] == 1:
      print(colSitioAnio [i]," - Cosechado en Octubre ")

      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][9])
      print("Desea restringir ", colSitioAnio[i], " desde Octubre ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Octubre")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Octubre 1 mes
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Octubre 2 meses
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Octubre 3 meses
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Octubre 4 meses
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i])
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Octubre 5 meses
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*2)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: #Octubre 6 meses
              matrizAreaRestriccion [i][9] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*3)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")


    #**********************************************************************************************



    # ******Bloqueos para Noviembre***************
    if matrizPrimerMesCosechado [i][10] == 1:
      print(colSitioAnio [i]," - Cosechado en Noviembre ")

      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][10])
      print("Desea restringir ", colSitioAnio[i], " desde Noviembre ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Noviembre")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Noviembre 1 mes
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Noviembre 2 meses
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Noviembre 3 meses
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i])
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Noviembre 4 meses
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*2)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Noviembre 5 meses
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*3)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Novimebre 6 meses
              matrizAreaRestriccion [i][10] = pesoMes*colAreaNeta[i]
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*4)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")



    #**********************************************************************************************

    # ******Bloqueos para Diciembre***************
    if matrizPrimerMesCosechado [i][11] == 1:
      print(colSitioAnio [i]," - Cosechado en Diciembre ")

      print("------------------------------------------------------------------------------------")
      print(colSitioAnio[i], "- Area Neta = ", vecAreaNeta [i], " - Tamaño -> ", vecRangosAreas[i], " - Area Cosechada -> ", matrizAreaRes[i][11])
      print("Desea restringir ", colSitioAnio[i], " desde Diciembre ")

      validador = False
      while validador == False:
        # El usuario escoge si desea o no restringir este sitio en el mes que se optimizo
        vecDeseaRestringir [i] = float(input("Si -> Ingrese 1  / No -> Ingrese 0"))
        if vecDeseaRestringir[i] == 1 or vecDeseaRestringir [i] == 0:
          validador = True

      validador1 = False
      pesoMes = 1
      if vecDeseaRestringir[i] == 1:
        # Si el usuario desea restringir a este sitio en el mes optimizado se pregunta en cuantos meses desea cosechar al sitio
        while validador1 == False:
          vecMesesRestriccion [i] = float(input("En cuantos meses desea cosechar"))
          if vecMesesRestriccion[i] >= 1 and vecMesesRestriccion[i] <= 6:
            print("Se cosechara ", colSitioAnio [i], "en ", vecMesesRestriccion[i], "meses desde Diciembre")
            #Segun el ingreso de cantidad de meses en que se desea cosechar se asgina las areas en la matriz de restricciones
            pesoMes = 1 / vecMesesRestriccion[i]

            #Calculo y asignacion de las areas a restringir dependiendo de lo ingresado
            if vecMesesRestriccion [i] == 1: # Diciembre 1 mes
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 2: # Diciembre 2 meses
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i])
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 3: # Diciembre 3 meses
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*2)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 4: #Diciembre 4 meses
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*3)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 5: # Diciembre 5 meses
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*4)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            if vecMesesRestriccion [i] == 6: # Novimebre 6 meses
              matrizAreaRestriccion [i][11] = pesoMes*colAreaNeta[i]
              print("Remanente para 2025 -> ", pesoMes*colAreaNeta[i]*5)
              valoresBloqueo()
              vecValidaAreaRestriccion[i] = 1

            validador1 = True
          if vecMesesRestriccion[i] < 1 or vecMesesRestriccion[i] > 6:
            print("No se puede cosechar en esa cantidad de meses")
      print("------------------------------------------------------------------------------------")

    #**********************************************************************************************


# ----------------------------------------------------------------------------------------------------------------------


#**********************Restriccion de Muisne segun AP (COMENTAR SI NO ES NECESARIO)*************************************
for i in range(columnas):
  # Encera todos los meses de Muisne
  matrizAreaRestriccion[38][i] = 0
  matrizAreaRestriccion[39][i] = 0
  matrizAreaRestriccion[40][i] = 0

#Define las areas que se van a cosechar
matrizAreaRestriccion[38][0] = 52.62
matrizAreaRestriccion[38][1] = 52.64
matrizAreaRestriccion[39][0] = 11
matrizAreaRestriccion[39][1] = 7.65
matrizAreaRestriccion[40][0] = 11
matrizAreaRestriccion[40][1] = 11.68

#Cambia a 1 el valor del vector para indicar que se debe restringir esas areas
vecValidaAreaRestriccion[38] = 1
vecValidaAreaRestriccion[39] = 1
vecValidaAreaRestriccion[40] = 1
#*************************************************************************************************************************

#Imprime la matriz de areas con las filas restringidas
#for i in range(filas):
#  print(colSitioAnio[i], matrizAreaRestriccion[i], vecValidaAreaRestriccion[i])


#Creacion un archivo de excel con las areas bloqueadas por sitio

#Titulos
titulosAreasRestringidas = ['Sitio-Año de plantación', 'Área ENE 2024', 'Área FEB 2024','Área MAR 2024', 'Área ABR 2024','Área MAY 2024','Área JUN 2024','Área JUL 2024','Área AGO 2024', 'Área SEP 2024','Área OCT 2024','Área NOV 2024','Área DIC 2024']

#Matriz para el df
numColAreaRest = columnas+1
matrizAreasRestringidas = [[0] * numColAreaRest for _ in range(filas)]

for j in range(columnas+1):
  for i in range(filas):
    if j==0:
      #Asigna los nombres de cada sitio-año de plantacion
      matrizAreasRestringidas[i][j] = colSitioAnio[i]
    if j>=1:
      #Asigna las areas restringidas
      matrizAreasRestringidas[i][j] = matrizAreaRestriccion [i][j-1]

#Creacion del data Frame con las areas restringidas
dataFrameAreasRest = pd.DataFrame(matrizAreasRestringidas, columns=titulosAreasRestringidas)

# Path donde se crea el archivo areas Restringidas
pathExcelAreasRest = '/content/drive/MyDrive/PB/Resultados/AreasRestringidas.xlsx'

#Guarda el data frame en el excel
dataFrameAreasRest.to_excel(pathExcelAreasRest, index=False)



1 ) Areas ENE  FC-022022 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
2 ) Areas ENE  FC-022022 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
3 ) Areas ENE  FC-022021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  11.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
4 ) Areas ENE  FC-032021 ->  0.0 ,  0.0 ,  0.0 ,  37.9 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
5 ) Areas ENE  FC-062021 ->  11.7 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  12.4 ,  8.0 ,  3.8 ,  2.9 ,  0.7 ,  2.8
6 ) Areas ENE  FC-112021 ->  0.0 ,  0.0 ,  28.2 ,  2.2 ,  0.0 ,  4.8 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
7 ) Areas ENE  FM-112021 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  22.4 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
8 ) Areas ENE  FM-182021 ->  5.8 ,  20.7 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
9 ) Areas ENE  MS-052022 ->  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0 ,  0.0
10 

KeyboardInterrupt: ignored

2da Ejecucion del modelo
(Incluye las areas restringidas)

In [ ]:

#****************************** MODELO DE OPTIMIZACION *********************************************************************************

#-------------  Definicion del modelo   -----------------
#optCosecha2 = pulp.LpProblem('OptimizacionCosecha2',pulp.LpMinimize) #Minimiza
optCosecha2 = pulp.LpProblem('OptimizacionCosecha2',pulp.LpMaximize) #Maximiza

# ------------VARIABLE DE DECISION (AREA A COSECHAR)-------------------
area_a_cosechar2 = [[pulp.LpVariable(f'x{i}_{j}', lowBound=0, cat='Continuous') for j in range(columnas)] for i in range(filas)]


#--------------FUNCION OBJETIVO-------------------------
#optCosecha2 += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar2[i][j] for j in range(columnas) for i in range(filas))  #Maximiza volumen total
optCosecha2 += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar2[i][j]*matrizPorcR8[i][j] for j in range(columnas) for i in range(filas))  #Maximiza volumen R8
#optCosecha2 += pulp.lpSum(colCostoActBio[i]*area_a_cosechar2[i][j] for j in range(columnas) for i in range(filas)) #Minimiza costos


#--------------RESTRICCIONES----------------------------
# 1) -- Restringe las areas a cosechar de cada sitio y mes para que no sean mayores al area neta
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    optCosecha2 += area_a_cosechar2[j][i] >= 0
    optCosecha2 += area_a_cosechar2[j][i] <= vecAreaNeta[j]
    optCosecha2 += area_a_cosechar2[j][i] <= areaMaxCosecha

# 2) Restringe las areas a cosechar de haciendas que estan retringidas por edad o accesibilidad (Para evitar errores en remanentes, evitar haciendas que tienen cero rendimiento y las cosecha sin dejarlas de remanente)
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    if matrizRendAux[j][i] > 0:
      optCosecha2 += area_a_cosechar2[j][i] >= 0
    if matrizRendAux[j][i] == 0:
      optCosecha2 += area_a_cosechar2[j][i] >=0
      optCosecha2 += area_a_cosechar2[j][i] <= 0


# 3) -- Restringe que la suma del area a cosecha de cada sitio en todo el año sea menor o igual al area neta
for j in range(filas): # recorre filas
  optCosecha2 += sum(area_a_cosechar2[j]) <= vecAreaNeta[j]


# 4) Restriccion de volumen anual
optCosecha2 += pulp.lpSum(matrizRendAux[i][j]*area_a_cosechar2[i][j] for j in range(columnas) for i in range(filas)) == volDes

# 5) Restringe los volumenes de cada mes dependiendo del volumen deseado
for j in range(columnas):
  optCosecha2 += sum(matrizRendAux[i][j]*area_a_cosechar2[i][j] for i in range(filas)) == vecVolDeseado[j]

# 6) Restringe los costos
#for j in range(columnas):
  #Restriccion de costos por kbft (actual a $90)
  #optCosecha2 += 1000*(sum(colCostoActBio[i]*area_a_cosechar2[i][j] for i in range(filas))) >= 106*(sum(matrizRendAux[i][j]*area_a_cosechar2[i][j] for i in range(filas)))
  #optCosecha2 += 1000*(sum(colCostoActBio[i]*area_a_cosechar2[i][j] for i in range(filas))) <= 118*(sum(matrizRendAux[i][j]*area_a_cosechar2[i][j] for i in range(filas)))

# 7) Restriccion de promedio densidad R8 y R11
#for j in range(columnas):
  #optCosecha2 += sum(matrizPromR8[i][j]*area_a_cosechar2[i][j] for i in range(filas)) >= 7.5 * sum(area_a_cosechar2[i][j] for i in range(filas))
  #optCosecha2 += sum(matrizPromR8[i][j]*area_a_cosechar2[i][j] for i in range(filas)) <= 8.5 * sum(area_a_cosechar2[i][j] for i in range(filas))
  #optCosecha2 += sum(matrizPromR11[i][j]*area_a_cosechar2[i][j] for i in range(filas)) >= 5.5 * sum(area_a_cosechar2[i][j] for i in range(filas))
  #optCosecha2 += sum(matrizPromR11[i][j]*area_a_cosechar2[i][j] for i in range(filas)) <= 6.5 * sum(area_a_cosechar2[i][j] for i in range(filas))

# 8) Restriccion de rendimientos ponderados mensuales
#Rendimientos ponderados deseados
#rendPondMaxDes2 = 15500
#rendPondMinDes2 = 12500
#Restriccion
#for j in range(columnas):
  #optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) >= rendPondMinDes2 * sum(area_a_cosechar[i][j] for i in range(filas))
  #optCosecha += sum(matrizRendAux[i][j]*area_a_cosechar[i][j] for i in range(filas)) <= rendPondMaxDes2 * sum(area_a_cosechar[i][j] for i in range(filas))


# 9) Restriccion de areas previamente optimizadas
for j in range(columnas):
  for i in range(filas):
    if vecValidaAreaRestriccion[i] == 1:
      optCosecha2 += area_a_cosechar2[i][j] == matrizAreaRestriccion[i][j]


#Resolucion
optCosecha2.solve()

#****************************** FIN MODELO*********************************************************************************

#--------------Impresion de resultados preliminares--------------------------------------------------------------------
a=1
for j in range(filas): # recorre filas
  # Imprime cada sitio con el area asignada en el mes que se va a cosechar
  print(a, ") Areas ENE ", colSitioAnio[j], "-> " ,round(area_a_cosechar2[j][0].value(),1), ", " ,round(area_a_cosechar2[j][1].value(),1), ", " ,round(area_a_cosechar2[j][2].value(),1),", " ,round(area_a_cosechar2[j][3].value(),1),", " ,round(area_a_cosechar2[j][4].value(),1),", " ,round(area_a_cosechar2[j][5].value(),1),", " ,round(area_a_cosechar2[j][6].value(),1),", " ,round(area_a_cosechar2[j][7].value(),1), ", " ,round(area_a_cosechar2[j][8].value(),1),", " ,round(area_a_cosechar2[j][9].value(),1),", " ,round(area_a_cosechar2[j][10].value(),1),", " ,round(area_a_cosechar2[j][11].value(),1))
  a = a+1

for b in range(columnas):
  #Imprime el volumen a entregar cada mes
  print("Vol> ", b+1, "->", round(sum(matrizRendAux[i][b]*area_a_cosechar2[i][b].value() for i in range(filas) ),0))

#Imprime el volumen total anual
print(sum(matrizRendAux[i][j]*area_a_cosechar2[i][j].value() for j in range(columnas) for i in range(filas) ))
#-----------------------------------------------------------------------------------------------------------------------



In [ ]:
matrizAreaRes2 = [[0] * columnas for _ in range(filas)]
countErrores2 = 0
for i in range(columnas): #recorre columnas
  for j in range(filas): # recorre filas
    matrizAreaRes2 [j][i] = area_a_cosechar2[j][i].value()
    if area_a_cosechar2[j][i].value() <-0.01:
      countErrores2 = countErrores2 + 1
print("Errores-> ", countErrores2)


#Vectores de resultados por mes
volTotalRes2 = []
areaTotalRes2 = []
rendTotalRes2 = []
costoTotalRes2 = []
vecCostoTotalResOpe2 = []
volTotalR8Res2 = []
volTotalR11Res2 = []
promR8Res2 = []
promR11Res2 = []

#------------CALCULO DE RESULTADOS POR MES -----------------------
#calculo de costos resultantes por mes
for j in range(columnas):
  costoTotalResul = 1000*(sum(colCostoActBio[i]*matrizAreaRes2[i][j] for i in range(filas))/sum(matrizRendAux[i][j]* matrizAreaRes2[i][j] for i in range(filas)))
  costoTotalRes2.append(round(costoTotalResul,1))


#Calculo de volumenes (total, R8, y R11) y area resultantes por mes
for j in range(columnas):
  # Volumen total por mes
  volTotal2 = sum(matrizRendAux[i][j]*matrizAreaRes2[i][j] for i in range(filas))
  volTotalRes2.append(round(volTotal2,0))
  #Area Total por mes
  areaTotal2 = sum(matrizAreaRes2[i][j] for i in range(filas))
  areaTotalRes2.append(round(areaTotal2,1))
  #Volumen R8 total por mes
  volTotalR82 = sum(matrizRendAux[i][j]*matrizAreaRes2[i][j]*matrizPorcR8[i][j] for i in range(filas))
  volTotalR8Res2.append(round(volTotalR82,0))
  #Volumen R11 total por mes
  volTotalR112 = sum(matrizRendAux[i][j]*matrizAreaRes2[i][j]*matrizPorcR11[i][j] for i in range(filas))
  volTotalR11Res2.append(round(volTotalR112,0))
  # Promedio R8 resultante por mes
  promR82 = sum(matrizAreaRes2[i][j]*matrizPromR8[i][j] for i in range(filas))
  promR8Res2.append(round(promR82/areaTotal2,1))
  # Promedio R11 resultante por mes
  promR112 = sum(matrizAreaRes2[i][j]*matrizPromR11[i][j] for i in range(filas))
  promR11Res2.append(round(promR112/areaTotal2,1))


#Rendimiento resultante por mes
for k in range(len(volTotalRes)):
  if volTotalRes2[k]>0:
    rendTotalRes2.append(round(volTotalRes2[k]/areaTotalRes2[k],1))
  else:
    rendTotalRes2.append(0)


#Presentacion de resultados
print("----------------------------------")
print('Volumen Anual Deseado')
print(sum(vecVolDeseado))
print("Volumen anual Resultante")
print(sum(volTotalRes2))
print('Volumen mensual deseado')
print(vecVolDeseado)
print("Volumen Resultante por mes")
print(volTotalRes2)
print("Volumen R8 por mes")
print(volTotalR8Res2)
print("Promedio R8 por mes")
print(promR8Res2)
print("Volumen R11 por mes")
print(volTotalR11Res2)
print("Promedio R11 por mes")
print(promR11Res2)
print("Area a cosechar por mes")
print(areaTotalRes2)
print("Costo kbft por mes")
print(costoTotalRes2)
#print("Costo kbft por mes INCLUYE OPERACIONAL")
#print(costoTotalResOpe)
print("Rendimiento por mes")
print(rendTotalRes2)
print("----------------------------------")

#-------------------------------------------------------------------------------------------


#----------------------------Creacion de Excel con resultados-------------------------------
columnasRes = 13+12+12
matrizResultados2 = [[0] * columnasRes for _ in range(filas)]

#Asignación de nombres de sitio a la matriz de resultados
for h in range(filas):
  matrizResultados2[h][0] = colSitioAnio[h]
k=1

# Asignación de resultados a la matriz
for k in range(columnasRes):
  for h in range(filas):
    if k>0 and k<=12:
      matrizResultados2[h][k] = area_a_cosechar2[h][k-1].value()
    if k>12 and k<=24:
      matrizResultados2[h][k] = matrizRendAux[h][k-1-12]
    if k>24:
      matrizResultados2[h][k] = matrizRendAux[h][k-1-24]*area_a_cosechar2[h][k-1-24].value()

for f in range(filas):
  print(matrizResultados2[f])

# Crea el Excel con los resultados,  areas por mes
#Nombres de las columnas del excel
nomColRes = ['Sitio-Año de plantación', 'Área ENE 2024', 'Área FEB 2024','Área MAR 2024', 'Área ABR 2024','Área MAY 2024','Área JUN 2024','Área JUL 2024','Área AGO 2024', 'Área SEP 2024','Área OCT 2024','Área NOV 2024','Área DIC 2024','Rendimiento ENE 2024','Rendimiento FEB 2024','Rendimiento MAR 2024','Rendimiento ABR 2024','Rendimiento MAY 2024','Rendimiento JUN 2024','Rendimiento JUL 2024','Rendimiento AGO 2024','Rendimiento SEP 2024','Rendimiento OCT 2024','Rendimiento NOV 2024','Rendimiento DIC 2024','VOL ENE 2024','VOL FEB 2024','VOL MAR 2024','VOL ABR 2024','VOL MAY 2024','VOL JUN 2024','VOL JUL 2024','VOL AGO 2024','VOL SEP 2024','VOL OCT 2024','VOL NOV 2024','VOL DIC 2024']
#Crea el data frame con los nombres de columnas y la matriz de resultados
df1 = pd.DataFrame(matrizResultados2, columns=nomColRes)
# Path donde se crea el archivo resOptimizacion
pathExcel = '/content/drive/MyDrive/PB/Resultados/resOptimizacion2.xlsx'
#Guarda el data frame en el excel
df1.to_excel(pathExcel, index=False)


#Creacion de matriz de remanentes.
colRem = 4
matrizRemanentes2 = [[0] * colRem for _ in range(filas)]
for j in range(filas):
  matrizRemanentes2[j][0] = colSitioAnio[j]
  matrizRemanentes2[j][1] = sum(matrizAreaRes2[j])
  matrizRemanentes2[j][2] = colAreaNeta[j]
  if colAreaNeta[j] - sum(matrizAreaRes2[j]) <0.01 and colAreaNeta[j] - sum(matrizAreaRes2[j]) > -0.01:
    matrizRemanentes2[j][3] = 0
  else:
    matrizRemanentes2[j][3] = colAreaNeta[j] - sum(matrizAreaRes2[j])



for i in range(filas):
  print("Sitio-","Cosechado-","Real-","Remanente")
  print(matrizRemanentes2[i][0],"->",round(matrizRemanentes2[i][1],1),"->",round(matrizRemanentes2[i][2],1),"->",round(matrizRemanentes2[i][3],1))
#Titulos de remanentes
nomMatRem = ['Sitio-Año de plantación','Area a cosechar', 'Area neta Sitio', 'Remanente']
#Crea el data frame de remanentes
df2 =pd.DataFrame(matrizRemanentes2, columns=nomMatRem)
pathExcel2 = '/content/drive/MyDrive/PB/Resultados/remanenteOptimizacion2.xlsx'
#Guarda el data frame en el excel
df2.to_excel(pathExcel2, index=False)
